In [1]:
from load_data import *
from utils import *
from training import *
from models import *

model = EncoderDecoder(34,10,256)
model.load_state_dict(torch.load("maml_yonc_256_5.weights"))

In [2]:
def flatten(lst):
    new_list = []
    for elt in lst:
        new_list = new_list + elt
        
    return new_list

In [3]:
def stringify_lst(lst):
    joined = ", ".join([str(x) for x in lst])
    return "[" + joined + "]"

In [4]:
from load_data import *
from utils import *
from training import *
from models import *

model = EncoderDecoder(34,10,256, recurrent_unit="GRU")
model.load_state_dict(torch.load("yonc_maml_gru_256_5.weights"))
model.set_dicts("a e i o u A E I O U b c d f g h j k l m n p q r s t v w x z .".split())


In [5]:
model(["za"])[1]

/Users/tommccoy/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[tensor([[[-3.1345e+01, -3.1722e+01, -1.0695e+01, -1.1993e+01, -1.9148e+01,
           -1.8033e+01, -1.8337e+01, -2.0455e+01, -1.7708e+01, -1.9106e+01,
           -1.7292e+01, -2.0588e+01, -2.1188e+01, -2.1301e+01, -1.7761e+01,
           -1.7994e+01, -2.0617e+01, -2.2182e+01, -2.0595e+01, -1.8950e+01,
           -2.4255e+01, -2.0938e+01, -1.6749e+01, -2.1373e+01, -1.9037e+01,
           -1.9733e+01, -2.0761e+01, -1.6086e+01, -1.9191e+01, -1.6779e+01,
           -2.0447e+01, -2.0326e+01, -8.7156e+00, -1.9322e-04]]],
        grad_fn=<LogSoftmaxBackward>),
 tensor([[[-2.5725e+01, -2.5668e+01, -1.1383e+01, -9.6359e+00, -1.6666e+01,
           -1.5934e+01, -1.6515e+01, -1.7916e+01, -1.5906e+01, -1.6670e+01,
           -1.5977e+01, -1.8716e+01, -1.9436e+01, -1.3169e+01, -9.7857e+00,
           -9.6532e+00, -1.2284e+01, -1.4586e+01, -1.2750e+01, -1.0180e+01,
           -1.6266e+01, -1.3031e+01, -9.9095e+00, -1.2121e+01, -1.0848e+01,
           -1.1269e+01, -1.2470e+01, -7.2635e+00, -1.1487e+

In [7]:
# Retry
uz = model.enc_lstm.wz_weights.transpose(0,1)[:10].data.numpy().transpose() # GOOD
wz = model.enc_lstm.wz_weights.transpose(0,1)[10:].data.numpy().transpose()
bz = model.enc_lstm.wz_bias.data.numpy() # GOOD

ur = model.enc_lstm.wr_weights.transpose(0,1)[:10].data.numpy().transpose()
wr = model.enc_lstm.wr_weights.transpose(0,1)[10:].data.numpy().transpose()
br = model.enc_lstm.wr_bias.data.numpy()

ux = model.enc_lstm.wx_weights.transpose(0,1).data.numpy().transpose() # GOOD
wx = model.enc_lstm.wrh_weights.transpose(0,1).data.numpy().transpose()
bx = model.enc_lstm.wx_bias.data.numpy() + model.enc_lstm.wrh_bias.data.numpy() # GOOD


full_w = flatten(np.concatenate([wz.transpose(), wr.transpose(), wx.transpose()], axis=1).tolist())
full_x = flatten(np.concatenate([uz, ur, ux], axis=0).transpose().tolist()) # CORRECT
full_b = flatten(np.concatenate([np.expand_dims(bz,1),np.expand_dims(br,1),np.expand_dims(bx,1)], axis=0).tolist()) # CORRECT

# No internal transpose:
# 0, no transpose: wrong
# 1, no transpose: right first thing, but not rest
# 0, transpose: wrong
# 1, transpose: wrong

# Internal transpose:
# 0, no transpose: wrong
# 1, no transpose: wrong
# 0, transpose: right first thing, but not rest
# 1, transpose: wrong


# Retry
uzd = model.dec_lstm.wz_weights.transpose(0,1)[:10].data.numpy().transpose() # GOOD
wzd = model.dec_lstm.wz_weights.transpose(0,1)[10:].data.numpy().transpose()
bzd = model.dec_lstm.wz_bias.data.numpy() # GOOD

urd = model.dec_lstm.wr_weights.transpose(0,1)[:10].data.numpy().transpose()
wrd = model.dec_lstm.wr_weights.transpose(0,1)[10:].data.numpy().transpose()
brd = model.dec_lstm.wr_bias.data.numpy()

uxd = model.dec_lstm.wx_weights.transpose(0,1).data.numpy().transpose() # GOOD
wxd = model.dec_lstm.wrh_weights.transpose(0,1).data.numpy().transpose()
bxd = model.dec_lstm.wx_bias.data.numpy() + model.dec_lstm.wrh_bias.data.numpy() # GOOD


full_wd = flatten(np.concatenate([wzd.transpose(), wrd.transpose(), wxd.transpose()], axis=1).tolist())
full_xd = flatten(np.concatenate([uzd, urd, uxd], axis=0).transpose().tolist()) # CORRECT
full_bd = flatten(np.concatenate([np.expand_dims(bzd,1),np.expand_dims(brd,1),np.expand_dims(bxd,1)], axis=0).tolist()) # CORRECT



tf_weights = open("tf_weights.js", "w")
tf_weights.write("emb_wg = " + stringify_lst(flatten(model.embedding.weights.data.numpy().tolist())) + ";\n")
tf_weights.write("full_x = " + stringify_lst(full_x) + ";\n")
tf_weights.write("full_w = " + stringify_lst(full_w) + ";\n")
tf_weights.write("full_b = " + stringify_lst(full_b) + ";\n")
tf_weights.write("full_xd = " + stringify_lst(full_xd) + ";\n")
tf_weights.write("full_wd = " + stringify_lst(full_wd) + ";\n")
tf_weights.write("full_bd = " + stringify_lst(full_bd) + ";\n")
tf_weights.write("out_wg = " + stringify_lst(flatten(model.dec_output.weights.transpose(0,1).data.numpy().tolist())) + ";\n")
tf_weights.write("out_wb = " + stringify_lst(model.dec_output.bias.data.numpy().tolist()) + ";\n")


748